In [ ]:
!pip3.10 install -U scikit-learn==1.5.2

In [ ]:
!pip3.10 install evaluate

In [ ]:
!pip3.10 install transformers

In [ ]:
!pip3.10 install -U acceleratehttp://is2.isa.ru:8096/notebooks/notebooks/storage/test_top2vec.ipynb#

In [ ]:
!pip3.10 install openpyxl

In [ ]:
!pip3.10 install nltk

In [ ]:
!nvidia-smi

In [ ]:
import nltk
nltk.download('punkt_tab')

## classification model

In [ ]:
import torch
import evaluate
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import DataCollatorWithPadding
import numpy as np

# Filter out all the "well-written" stuff

In [ ]:
import torch
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from nltk.tokenize import sent_tokenize
from tqdm import tqdm, trange
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

accuracy = evaluate.load("accuracy")

model = AutoModelForSequenceClassification.from_pretrained(
    "segm_model/my_model", num_labels=4)

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
#    return accuracy.compute(predictions=predictions, references=labels)
    return f1.compute(predictions=predictions, references=labels, average='macro')


tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors = 'pt')

training_args = TrainingArguments(
    output_dir="segm_model/my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
    #no_cuda=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    #train_dataset=tokenized_ds["train"],
    #eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

def remove_trash(txt_):
    txts = []
    texts = sent_tokenize(txt_)
    for txt in texts:
        tokens = tokenizer(txt, truncation=True)
        predictions = trainer.predict([tokens])
        lbl = np.argmax(predictions.predictions, axis=-1)
        if lbl != 1:
            txts.append(txt)
    if len(txts) == 0:
        return txt_
    else:
        return " ".join(txts) 

def preprocess_reviews(fname): 
    df = pd.read_excel(fname)
    dat = []

    for i, row in df.iterrows():
        dat.append([i,row['rate'], remove_trash(row["review"])])

    return pd.DataFrame(dat,columns=("id","score","review"))

df = preprocess_reviews("tp_2017conference.xlsx")

del model

df.to_csv("preprocessed_rewies.csv",sep=";")

# Do classifiction

In [ ]:
import pandas as pd

df = pd.read_csv("preprocessed_rewies.csv",sep=";")

In [ ]:
df

In [ ]:
import torch
import evaluate
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import DataCollatorWithPadding
import numpy as np

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

def preprocess_function(examples):
    return tokenizer([str(e) for e in examples["text"]], truncation=True, max_length=512)


is_new_model = AutoModelForSequenceClassification.from_pretrained(
    "is_new_modelxlm-roberta-large/checkpoint-50", num_labels=2).to(device)

data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding='longest',
        max_length=512,
        pad_to_multiple_of=8,
        return_tensors='pt',
    )

training_args = TrainingArguments(
    output_dir="is_new_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    #eval_strategy="epoch",
    #save_strategy="epoch",
    #load_best_model_at_end=True,
    push_to_hub=False,
    #no_cuda = True
)

trainer = Trainer(
   model=is_new_model,
   args=training_args,
   #train_dataset=tokenized_ds["train"],
   #eval_dataset=tokenized_ds["test"],    
   tokenizer=tokenizer,
   data_collator=data_collator,
   #compute_metrics=compute_metrics,
)


for dataset_type in ["pure","question","is_new"]:

    high_score_lens = []
    med_score_lens = []
    low_score_lens = []


    #df = pd.read_excel("tp_2017conference.xlsx")
    dat = []
    for i, row in df.iterrows():
        score = int(row['score'][row['score'].rfind("#")+1:row["score"].rfind(":")])
        qtags = row["review"].count("?")
        text = row["review"]
        if dataset_type == "is_new":
            tokens = tokenizer(text, truncation=True)
            predictions = trainer.predict([tokens])
            if np.argmax(predictions.predictions, axis=-1).sum() == 0:     
                text = "IS_NEW" + " " + text                                  
        if score > 6:
            score = "high"
            high_score_lens.append(row["review"].count("?"))
        else:
            if score > 4:
                score = "medium"
            else:
                score = "low"
                med_score_lens.append(row["review"].count("?"))
        if dataset_type == "is_new" or  dataset_type == "question":
            if qtags > 1:
                text = "MANY_QUESTIONS" + " " + text    
        dat.append([text,score])

    dataset = pd.DataFrame(dat,columns=["text","label"])
    dataset.to_csv("cls_dataset.csv",sep=";")
    
    train_data = []
    
    if dataset_type == "is_new":
        del is_new_model
        del trainer

    for model_name in ['xlm-roberta-large','xlm-roberta-base','roberta-base','roberta-large','distilbert-base-uncased','distilroberta-base','bert-base-uncased','bert-large-uncased']:
        dataset = pd.read_csv("cls_dataset.csv",sep=";")

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")

        def preprocess_function(examples):
            return tokenizer([str(e) for e in examples["text"]], truncation=True, max_length=512)
        encoder = LabelEncoder()
        encoder.fit(dataset['label'])

        cls_model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=len(encoder.classes_))

        dtrain, dtest = train_test_split(dataset, test_size=0.2, train_size=0.8, shuffle = True)  

        dtrain = Dataset.from_pandas(dtrain)
        dtest = Dataset.from_pandas(dtest)

        dataset = DatasetDict({"train": dtrain,"test":dtest})

        def map_labels(example):
            return {
                'label': encoder.transform([example['label']])[0]
            }

        dataset = dataset.map(map_labels, num_proc=16)

        tokenized_ds = dataset.map(preprocess_function, batched=True).remove_columns("text").with_format("torch")
        data_collator = DataCollatorWithPadding(
                tokenizer=tokenizer,
                padding='longest',
                max_length=512,
                pad_to_multiple_of=8,
                return_tensors='pt',
            )

        accuracy = evaluate.load("accuracy")


        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, -1)
            return accuracy.compute(predictions=predictions, references=labels)

        training_args = TrainingArguments(
            output_dir="review_clf_model2" + dataset_type + model_name,
            learning_rate=2e-5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=20,
            weight_decay=0.01, 
            warmup_ratio=0.15,
            max_grad_norm=1.0,    
            eval_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,
            load_best_model_at_end=True,
            push_to_hub=False,
            #no_cuda=True
        )

        trainer = Trainer(
           model=cls_model,
           args=training_args,
           train_dataset=tokenized_ds["train"],
           eval_dataset=tokenized_ds["test"],    
           tokenizer=tokenizer,
           data_collator=data_collator,
           compute_metrics=compute_metrics,
        )

        trainer.train()
        predictions = trainer.predict(tokenized_ds["test"])
        preds = np.argmax(predictions.predictions, axis=-1)
        prec = evaluate.load("precision")
        rec = evaluate.load("recall")
        f1 = evaluate.load("f1")
        p = prec.compute(predictions=preds, references=predictions.label_ids,average='macro')['precision']
        r = rec.compute(predictions=preds, references=predictions.label_ids,average='macro')['recall']
        f1 = f1.compute(predictions=preds, references=predictions.label_ids,average='macro')['f1']

        print(model_name,p,r,f1)
        train_data.append([model_name,p,r,f1])

    pd.DataFrame(train_data, columns=['Model','P','R','F1']).to_csv("review_classifier2" + dataset_type + ".csv",sep=";")     
    # del trainer
    # del is_new_model